In [7]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

In [2]:
# Define las credenciales de la base de datos
username = 'root'
password = 'MySQL'
database_name = 'prueba_globant'
host = 'localhost'  # o la IP si es remoto
port = '3306'  # Puerto por defecto de MySQL

In [3]:
# Crea la URL de conexión para SQLAlchemy
database_url = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}'
# Crea el motor de conexión a la base de datos
engine = create_engine(database_url)

In [4]:
hired_employees_df = pd.read_excel('C:/Users/franc/Downloads/hired_employees.xlsx', header=None, names=['id', 'name', 'datetime', 'department_id', 'job_id'])

In [17]:
def convert_to_mysql_format(iso_datetime_str):
    if isinstance(iso_datetime_str, str):
        try:
            datetime_obj = datetime.fromisoformat(iso_datetime_str.rstrip('Z'))
            return datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            return '1000-01-01 00:00:00'
    else:
        return '1000-01-01 00:00:00'

In [26]:
import pandas as pd

def convert_float_columns_to_int(df, columns_to_convert):
    """
    Convierte las columnas especificadas de un DataFrame de pandas de float a int.

    Parámetros:
    - df: DataFrame de pandas.
    - columns_to_convert: Lista de nombres de columnas para convertir a enteros.

    Retorna:
    - DataFrame con las columnas especificadas convertidas a tipo int.
    """
    for column in columns_to_convert:
        # Comprueba si la columna está en el DataFrame y si es de tipo float
        if column in df.columns and pd.api.types.is_float_dtype(df[column]):
            # Convierte la columna a int
            # Usamos `astype('Int64')` para manejar correctamente los NaNs, 
            # convirtiéndolos a valores enteros NA de pandas.
            df[column] = df[column].astype('Int64')
    return df


In [27]:
columns_to_convert = ['id', 'department_id', 'job_id']
convert_float_columns_to_int(hired_employees_df, columns_to_convert)

,id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07 02:48:42,2,96
1,2,Ty Hofer,2021-05-30 05:43:46,8,<NA>
2,3,Lyman Hadye,2021-09-01 23:27:38,5,52
3,4,Lotti Crowthe,2021-10-01 13:04:21,12,71
4,5,Gretna Lording,2021-10-10 22:22:17,6,80
...,...,...,...,...,...
1994,1995,Goran Yong,2021-08-22 21:57:53,2,83
1995,1996,Cirstoforo Youings,2021-04-01 17:48:42,3,23
1996,1997,Wilek Yurkevich,2021-05-04 12:19:50,2,132
1997,1998,Jerry Yven,2021-10-03 14:12:50,7,100


In [25]:
hired_employees_df['datetime'] = hired_employees_df['datetime'].apply(convert_to_mysql_format)

In [21]:
hired_employees_df['name'].fillna('Unknown', inplace=True)


In [22]:
hired_employees_df.head()

,id,name,datetime,department_id,job_id
0,1.0,Harold Vogt,2021-11-07 02:48:42,2.0,96.0
1,2.0,Ty Hofer,2021-05-30 05:43:46,8.0,NaN
2,3.0,Lyman Hadye,2021-09-01 23:27:38,5.0,52.0
3,4.0,Lotti Crowthe,2021-10-01 13:04:21,12.0,71.0
4,5.0,Gretna Lording,2021-10-10 22:22:17,6.0,80.0


In [33]:
hired_employees_df.to_sql('hired_employees', con=engine, if_exists='append', index=False)

1999